In [52]:
import pandas as pd
import requests
import time

In [53]:
#get data from f1_race_data_all and save to variable

data = pd.read_csv('f1_race_data_all.csv')
data.head()

,driver_number,broadcast_name,full_name,first_name,last_name,name_acronym,country_code_x,team_name,meeting_key,session_key,...,country_name,country_code_y,circuit_short_name,qualifying_position_x,starting_position,qualifying_position_y,final_position,wins_before,pit_stops_count,avg_pit_stop_duration
0,1,M VERSTAPPEN,Max VERSTAPPEN,Max,Verstappen,VER,NED,Red Bull Racing,1142,7779,...,Saudi Arabia,KSA,Jeddah,15,15,1,2,1,0,0.0
1,2,L SARGEANT,Logan SARGEANT,Logan,Sargeant,SAR,USA,Williams,1142,7779,...,Saudi Arabia,KSA,Jeddah,20,20,16,16,0,0,0.0
2,4,L NORRIS,Lando NORRIS,Lando,Norris,NOR,GBR,McLaren,1142,7779,...,Saudi Arabia,KSA,Jeddah,19,19,7,17,0,0,0.0
3,10,P GASLY,Pierre GASLY,Pierre,Gasly,GAS,FRA,Alpine,1142,7779,...,Saudi Arabia,KSA,Jeddah,10,9,9,9,0,0,0.0
4,11,S PEREZ,Sergio PEREZ,Sergio,Perez,PER,MEX,Red Bull Racing,1142,7779,...,Saudi Arabia,KSA,Jeddah,1,1,2,1,0,0,0.0


In [54]:
# pobierz tylko session_keys z data i zapisz do all_sessions_df
all_sessions_df = data[['session_key']].drop_duplicates().reset_index(drop=True)
all_sessions_df.head()

,session_key
0,7779
1,7787
2,7953
3,9069
4,9070


In [55]:
# pobierz kierowcow z df_all_sessions_df i zapisz do all_drivers_df
all_drivers_df = data[['driver_number','broadcast_name']].drop_duplicates().reset_index(drop=True)
all_drivers_df.head()

,driver_number,broadcast_name
0,1,M VERSTAPPEN
1,2,L SARGEANT
2,4,L NORRIS
3,10,P GASLY
4,11,S PEREZ


In [56]:
# pobierz rodzaje opon 
# https://api.openf1.org/v1/stints?session_key={session_key}
#     "compound": "SOFT",
#     "driver_number": 16,
#     "lap_end": 20,
#     "lap_start": 1,
#     "meeting_key": 1219,
#     "session_key": 9165,
#     "stint_number": 1,
#     "tyre_age_at_start": 3

def get_tyre_stints(session_keys):
    all_tyres = []
    
    for session_key in session_keys:
        url = f"https://api.openf1.org/v1/stints?session_key={session_key}"
        response = requests.get(url)
        
        if response.status_code == 200:
            data = response.json()
            
            if isinstance(data, list) and len(data) > 0:
                for stint in data:
                    tyre_record = {
                        'session_key': session_key,
                        'driver_number': stint.get('driver_number'),
                        'stint_number': stint.get('stint_number'),
                        'compound': stint.get('compound'),
                        'lap_start': stint.get('lap_start'),
                        'lap_end': stint.get('lap_end'),
                        'tyre_age_at_start': stint.get('tyre_age_at_start'),
                        'meeting_key': stint.get('meeting_key')
                    }
                    all_tyres.append(tyre_record)
                
                print(f"Session {session_key}: Pobrano {len(data)} stints")
            else:
                print(f"Session {session_key}: Brak danych o oponach")
        else:
            print(f"Error fetching tyre stints for session {session_key}: {response.status_code}")
        
        sleep(0.5)
    
    return pd.DataFrame(all_tyres)

In [ ]:
# get tyres stints for all sessions
from time import sleep
tyres_stints_df = get_tyre_stints(all_sessions_df['session_key'].tolist())
tyres_stints_df.head()

Session 7779: Pobrano 44 stints
Session 7787: Pobrano 85 stints
Session 7953: Pobrano 70 stints
Session 9069: Pobrano 23 stints
Session 9070: Pobrano 43 stints
Session 9078: Pobrano 40 stints
Session 9094: Pobrano 57 stints
Session 9102: Pobrano 63 stints
Session 9110: Pobrano 53 stints
Session 9117: Pobrano 31 stints
Session 9118: Pobrano 83 stints
Session 9126: Pobrano 44 stints
Session 9133: Pobrano 56 stints
Session 9140: Pobrano 42 stints
Session 9141: Pobrano 57 stints
Session 9149: Pobrano 121 stints
Session 9157: Pobrano 45 stints
Session 9165: Pobrano 44 stints
Session 9173: Pobrano 63 stints
Session 9181: Pobrano 58 stints
Session 9189: Pobrano 51 stints
Session 9197: Pobrano 57 stints
Session 9204: Pobrano 20 stints
Session 9205: Pobrano 87 stints
Session 9212: Pobrano 20 stints
Session 9213: Pobrano 56 stints
Session 9220: Pobrano 23 stints
Session 9221: Pobrano 73 stints
Session 9472: Pobrano 63 stints
Session 9480: Pobrano 39 stints
Session 9488: Pobrano 55 stints
Session

In [ ]:
# dla kazdego zwawodnika w kazdej sesji oblicz ile stints mial
tyres_stints_count_df = tyres_stints_df.groupby(['session_key', 'driver_number']).size().reset_index(name='num_stints')
tyres_stints_count_df.head()

,session_key,driver_number,num_stints
0,7779,1,2
1,7779,2,2
2,7779,4,3
3,7779,10,2
4,7779,11,2


In [ ]:
# w jednej kolumnie dla kazdego zawodnika i kazdej sesji podaj jakie mial kolejno rodzaje opon
tyres_stints_list_df = tyres_stints_df.groupby(['session_key', 'driver_number'])['compound'].apply(list).reset_index(name='tyre_compounds')
tyres_stints_list_df.head()

,session_key,driver_number,tyre_compounds
0,7779,1,"[MEDIUM, HARD]"
1,7779,2,"[HARD, MEDIUM]"
2,7779,4,"[SOFT, HARD, MEDIUM]"
3,7779,10,"[MEDIUM, HARD]"
4,7779,11,"[MEDIUM, HARD]"


In [ ]:
# Załóżmy, że masz tyres_stints_list_df z kolumną 'tyre_compounds'
for compound in ['SOFT', 'MEDIUM', 'HARD']:
    tyres_stints_list_df[f'used_{compound.lower()}'] = tyres_stints_list_df['tyre_compounds'].apply(lambda x: int(compound in x))

#drop tyre_compounds
tyres_stints_list_df = tyres_stints_list_df.drop(columns=['tyre_compounds'])
tyres_stints_list_df.head()

,session_key,driver_number,used_soft,used_medium,used_hard
0,7779,1,0,1,1
1,7779,2,0,1,1
2,7779,4,1,1,1
3,7779,10,0,1,1
4,7779,11,0,1,1


In [ ]:
# zrob merge data and tyres_stints_count_df on session_key and driver_number
data = data.merge(tyres_stints_count_df, on=['session_key', 'driver_number'], how='left')
data = data.merge(tyres_stints_list_df, on=['session_key', 'driver_number'], how='left')
data.head()

,driver_number,broadcast_name,full_name,first_name,last_name,name_acronym,country_code_x,team_name,meeting_key,session_key,...,starting_position,qualifying_position_y,final_position,wins_before,pit_stops_count,avg_pit_stop_duration,num_stints,used_soft,used_medium,used_hard
0,1,M VERSTAPPEN,Max VERSTAPPEN,Max,Verstappen,VER,NED,Red Bull Racing,1142,7779,...,15,1,2,1,0,0.0,2.0,0.0,1.0,1.0
1,2,L SARGEANT,Logan SARGEANT,Logan,Sargeant,SAR,USA,Williams,1142,7779,...,20,16,16,0,0,0.0,2.0,0.0,1.0,1.0
2,4,L NORRIS,Lando NORRIS,Lando,Norris,NOR,GBR,McLaren,1142,7779,...,19,7,17,0,0,0.0,3.0,1.0,1.0,1.0
3,10,P GASLY,Pierre GASLY,Pierre,Gasly,GAS,FRA,Alpine,1142,7779,...,9,9,9,0,0,0.0,2.0,0.0,1.0,1.0
4,11,S PEREZ,Sergio PEREZ,Sergio,Perez,PER,MEX,Red Bull Racing,1142,7779,...,1,2,1,0,0,0.0,2.0,0.0,1.0,1.0


In [ ]:
# convert used_soft, used_medium, used_hard to int
data['used_soft'] = data['used_soft'].fillna(0).astype(int)
data['used_medium'] = data['used_medium'].fillna(0).astype(int)
data['used_hard'] = data['used_hard'].fillna(0).astype(int)
data['num_stints'] = data['num_stints'].fillna(0).astype(int)
data.head()

,driver_number,broadcast_name,full_name,first_name,last_name,name_acronym,country_code_x,team_name,meeting_key,session_key,...,starting_position,qualifying_position_y,final_position,wins_before,pit_stops_count,avg_pit_stop_duration,num_stints,used_soft,used_medium,used_hard
0,1,M VERSTAPPEN,Max VERSTAPPEN,Max,Verstappen,VER,NED,Red Bull Racing,1142,7779,...,15,1,2,1,0,0.0,2.0,0,1,1
1,2,L SARGEANT,Logan SARGEANT,Logan,Sargeant,SAR,USA,Williams,1142,7779,...,20,16,16,0,0,0.0,2.0,0,1,1
2,4,L NORRIS,Lando NORRIS,Lando,Norris,NOR,GBR,McLaren,1142,7779,...,19,7,17,0,0,0.0,3.0,1,1,1
3,10,P GASLY,Pierre GASLY,Pierre,Gasly,GAS,FRA,Alpine,1142,7779,...,9,9,9,0,0,0.0,2.0,0,1,1
4,11,S PEREZ,Sergio PEREZ,Sergio,Perez,PER,MEX,Red Bull Racing,1142,7779,...,1,2,1,0,0,0.0,2.0,0,1,1
